In [1]:
import torch
import torchvision
from torchvision import datasets, models, transforms
from torchvision.io import ImageReadMode
from torchvision.io import read_image
from torchvision.datasets import ImageFolder
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import pandas as pd
import torch 
import matplotlib.pyplot as plt
from scipy.io import loadmat
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.nn.functional as F
import os
from torchvision import transforms as T
import math
from sklearn.metrics import *

In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [3]:
train = pd.read_csv('train_image.csv')
test = pd.read_csv('val_image.csv')
print(train)

     Recording  Label
0        A4363      6
1        A4447      4
2        A0822      5
3        A0415      2
4        A5712      1
...        ...    ...
6184     A3728      5
6185     A0683      1
6186     A6561      5
6187     A5674      2
6188     A3270      3

[6189 rows x 2 columns]


In [4]:
class ECGimage_DB(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir 
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0] +'.png')
        image = read_image(img_path, mode=ImageReadMode.RGB)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image.float(), label-1

In [5]:
class ECGimage_DBv(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir 
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0] +'.png')
        image = read_image(img_path, mode=ImageReadMode.RGB)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image.float(), label-1

In [6]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 9)
model = model.to(device)
learning_rate = 1e-3
epochs = 5
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

/home/ubuntu/miniconda3/envs/IECG_class/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ubuntu/miniconda3/envs/IECG_class/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [8]:
path="ECGimg_ts"
train_db=ECGimage_DB('train_image.csv',path)
valid_db= ECGimage_DBv('val_image.csv',path)

train_dl=DataLoader(train_db, batch_size=16)
valid_dl=DataLoader(valid_db, batch_size=16)
# image = read_image(os.path.join(path,'A0001.png'),mode=ImageReadMode.RGB)

# print(image.size())
# flatten = nn.Flatten()
# flat_image = flatten(image).float()
# print(flat_image.size())

In [9]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader,0):
        X = X.to(device)
        y = y.to(device)
        # Backpropagation
        pred = model(X)
        optimizer.zero_grad()
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [10]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    running_y, running_pred = [], []

    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            pred1 = torch.argmax(pred,1)
            labels = list(y.cpu().numpy())
            
            preds = list(pred1.cpu().numpy())
            
            running_y.extend(labels)
            running_pred.extend(preds)
            test_loss += loss_fn(pred, y).item()
            # correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    f1 = f1_score(running_y,running_pred, average = 'macro')

    test_loss /= num_batches
    correct /= size
    # print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    print(f"Test Error: \n F1: {(f1):>0.1f}, Avg loss: {test_loss:>8f} \n")

In [11]:
epochs = 50
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dl, model, loss_fn, optimizer)
    test_loop(valid_dl, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.414014  [    0/ 6189]
loss: 1.856285  [ 1600/ 6189]
loss: 1.743071  [ 3200/ 6189]
loss: 1.969655  [ 4800/ 6189]
Test Error: 
 F1: 0.2, Avg loss: 1.635721 

Epoch 2
-------------------------------
loss: 2.043333  [    0/ 6189]
loss: 1.646938  [ 1600/ 6189]
loss: 1.487276  [ 3200/ 6189]
loss: 1.804085  [ 4800/ 6189]
Test Error: 
 F1: 0.3, Avg loss: 1.480400 

Epoch 3
-------------------------------
loss: 1.855991  [    0/ 6189]
loss: 1.455943  [ 1600/ 6189]
loss: 1.207871  [ 3200/ 6189]
loss: 1.563575  [ 4800/ 6189]
Test Error: 
 F1: 0.4, Avg loss: 1.430946 

Epoch 4
-------------------------------
loss: 1.719898  [    0/ 6189]
loss: 1.309095  [ 1600/ 6189]
loss: 1.218495  [ 3200/ 6189]
loss: 1.629312  [ 4800/ 6189]
Test Error: 
 F1: 0.4, Avg loss: 1.391986 

Epoch 5
-------------------------------
loss: 1.639001  [    0/ 6189]
loss: 1.313859  [ 1600/ 6189]
loss: 1.138521  [ 3200/ 6189]
loss: 1.432736  [ 4800/ 6189]
Test Error: 
 F1: 0.4, A